In [0]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import seaborn as sns
	
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier




In [0]:
df_train_data = pd.read_csv("train_data.csv",header=None)
df_train_labels = pd.read_csv("train_labels.csv",header=None)
df_test_data = pd.read_csv("test_data.csv",header=None)

columnsD = []
for i in range(1,df_train_data.shape[1]+1):
    txt = ""
    if i <= 24:
        txt = "mean_band_"+str(i)        
    elif i <= 24*2:
        txt = "median_band_"+str(i-24)
    elif i <= 24*3:
        txt = "variance_band_"+str(i-24*2)
    elif i <= 24*4:
        txt = "kurtosis_band_"+str(i-24*3)
    elif i <= 24*5:
        txt = "skewness_band_"+str(i-24*4)
    elif i <= 24*6:
        txt = "min_band_"+str(i-24*5)
    elif i <= 24*7:
        txt = "max_band_"+str(i-24*6)
    elif i <= (24*7)+12:
        txt = "mean_pc_"+str(i-((24*7)))
    elif i <= (24*7)+(12*2):
        txt = "stan_deriv_pc_"+str(i-((24*7)+12))
    elif i <= (24*7)+(12*3):
        txt = "min_pc_"+str(i-((24*7)+(12*2)))
    elif i <= (24*7)+(12*4):
        txt = "max_pc_"+str(i-((24*7)+(12*3)))
    elif i <= (24*7)+(12*4)+12:
        txt = "mean_coef_"+str(i-((24*7)+(12*4)))
    elif i <= (24*7)+(12*4)+(12*2):
        txt = "stan_deriv_coef_"+str(i-((24*7)+(12*4)+12))
    elif i <= (24*7)+(12*4)+(12*3):
        txt = "min_coef_"+str(i-((24*7)+(12*4)+(12*2)))
    elif i <= (24*7)+(12*4)+(12*4):
        txt = "max_coef_"+str(i-((24*7)+(12*4)+(12*3)))
    
        
    columnsD.append(txt)
    
df_test_data.columns = columnsD
df_train_data.columns = columnsD

#In the future it can be expanded to more features
columnsL = []
for i in range(1,df_train_labels.shape[1]+1):
    columnsL.append("Column_"+str(i))
df_train_labels.columns = columnsL


df_train_labels["Column_1"] = df_train_labels["Column_1"].astype('category')







In [3]:
union = pd.concat([df_train_data, df_train_labels ], axis = 1)
info = union.Column_1.value_counts()
result = pd.DataFrame()
print(info)
info[10]
for cat in range(1,11):
  
    data_to_res = union[union.Column_1 == cat]
          #print(data_to_res)
    samples = info[cat]
    if(info[cat] < 500):
        samples = 500
    if(cat == 1):
        samples = 1000
    df = resample(data_to_res, replace=True, n_samples=samples)

    result = pd.concat([result,df])


print(result.Column_1.value_counts())
result = result.sample(n=result.shape[0])

df_train_labels = pd.DataFrame(result["Column_1"])

df_train_labels.columns = columnsL


df_train_labels["Column_1"] = df_train_labels["Column_1"].astype('category')
df_train_data = pd.DataFrame(result[columnsD])


1     2178
2      618
3      326
6      260
4      253
5      214
8      195
7      141
9       92
10      86
Name: Column_1, dtype: int64
1     1000
2      618
10     500
9      500
8      500
7      500
6      500
5      500
4      500
3      500
Name: Column_1, dtype: int64


# Creating Training and Test Sets 

In [0]:
X = df_train_data.values
y = df_train_labels.values.reshape(len(df_train_labels))
X_test_unlabeled = df_test_data.values







# Scaling data

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X= scaler.transform(X)
X_test_unlabeled = scaler.transform(X_test_unlabeled)

# Grid Search of different Models

In [0]:


from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=6, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.metrics import log_loss




from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier



models1 = {
    'XGBClassifier': XGBClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'LogisticRegression' : LogisticRegression(),
    'SVC': SVC(class_weight='balanced', probability=True)
}

params1 = {
    'XGBClassifier': { 'n_estimators': [256], 'learning_rate': [0.2,0.8, 1.0] },
    'RandomForestClassifier': { 'n_estimators': [256] },
    'AdaBoostClassifier':  { 'n_estimators': [256] },
    'GradientBoostingClassifier': { 'n_estimators': [256], 'learning_rate': [0.2,0.8, 1.0] },
    'LogisticRegression': {'penalty': ['l2'], 'C' : [0.01,0.1,1]},
    'SVC': {'kernel': ['linear'], 'C': [ 1, 3, 5,15]}
}

In [8]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X, y, scoring='accuracy', n_jobs=8)

Running GridSearchCV for XGBClassifier.
Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed:  1.8min finished


Running GridSearchCV for AdaBoostClassifier.
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed: 37.6min remaining: 47.0min
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed: 40.8min finished


Running GridSearchCV for LogisticRegression.
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:   49.6s remaining:  1.0min
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:  1.5min finished


Running GridSearchCV for SVC.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=8)]: Done  10 out of  12 | elapsed:  6.9min remaining:  1.4min
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:  7.5min finished


In [9]:
helper1.score_summary(sort_by='max_score')

XGBClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
LogisticRegression
SVC


,estimator,min_score,mean_score,max_score,std_score,C,kernel,learning_rate,n_estimators,penalty
1,XGBClassifier,0.860874,0.870079,0.881628,0.00863359,NaN,NaN,0.8,256,NaN
0,XGBClassifier,0.870469,0.875939,0.878933,0.00387361,NaN,NaN,0.2,256,NaN
3,RandomForestClassifier,0.871002,0.873806,0.878415,0.00328442,NaN,NaN,NaN,256,NaN
5,GradientBoostingClassifier,0.868267,0.872028,0.878415,0.0045397,NaN,NaN,0.2,256,NaN
2,XGBClassifier,0.865139,0.865967,0.866667,0.000630413,NaN,NaN,1,256,NaN
6,GradientBoostingClassifier,0.828358,0.842501,0.86181,0.0141371,NaN,NaN,0.8,256,NaN
14,SVC,0.841067,0.848887,0.854847,0.00577755,15,linear,NaN,NaN,NaN
13,SVC,0.843817,0.847465,0.854312,0.00484485,5,linear,NaN,NaN,NaN
12,SVC,0.842217,0.844792,0.849491,0.00332803,3,linear,NaN,NaN,NaN
11,SVC,0.836267,0.84087,0.84435,0.00339406,1,linear,NaN,NaN,NaN


## Stacked Classification using probabilities

In [0]:
clf1 = GradientBoostingClassifier(learning_rate=0.2, n_estimators = 256)
clf2 = XGBClassifier(learning_rate = 0.2, n_estimators = 256)
clf3 = RandomForestClassifier(n_estimators = 256)
clf4 = SVC(C=15, kernel='linear',probability=True,class_weight='balanced')


from sklearn.ensemble import VotingClassifier
sclf = VotingClassifier(estimators=[
    ('gr', clf1), ('xgb', clf2), ('rf', clf3),('svc', clf4)],
                         voting='soft')

sclf = sclf.fit(X, y)



## Stacked Classification and GridSearch

### Save best model pipeline

In [0]:
name_best_model = 'stacked_LR_XGB_SVC'


from sklearn.model_selection import train_test_split



X_train = X
y_train = y
X_test = X
y_test = y

best_model= sclf 

## Semi-supervised learning (Pseudo Labeling)

In [0]:
y_test_unlabeled = best_model.predict(X_test_unlabeled)

### Concatenate training labels with test set pseudo labels

In [0]:
y_semisupervised = np.concatenate((y, y_test_unlabeled))

### Concatenate training features with test features

In [0]:
X_semisupervised = np.concatenate((X, X_test_unlabeled))

### Train the model including pseudo labels

In [0]:

best_model_ss= best_model.fit(X_semisupervised,y_semisupervised)
accuracy_best_model = best_model_ss.score(X_test,y_test)


print('Cross Validation Accuracy of {} (best classifier) with SEMI-SUPERVISED LEARNING: {:.3f}'
       .format(name_best_model,accuracy_best_model))

  

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y, best_model.predict(X)))